# The PAP algorithm (using the simplified XPP)

Pouchou, J.-L., & Pichoir, F. (1991). Quantitative Analysis of Homogeneous or Stratified Microvolumes Applying the Model “PAP.”

https://sci-hub.ru/10.1007/978-1-4899-2617-3_4



In [ ]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go
import pandas as pd

In [ ]:
def theoretical_energy(line: str):
    "Takes a X-ray line and gives the energy in keV."
    element = line.split('_')[0]
    line_name = line.split('_')[1]
    return hs.material.elements[element]['Atomic_properties']['Xray_lines'][line_name]['energy (keV)']

#### Area of $\phi(\rho z)$

Primary intentisty (PI) = R/S

$ F = \int \limits_{0}^{\inf} \phi(\rho z)d(\rho z) = (R/S) \cdot Q_l^A(E_0)$


We use an approximation for the ionization cross-section, and calculate R and 1/S as below.


# Area F of the distribution $\phi(\rho z)$

## Deceleation of electrons ( $1/S$ )

$ 1 / S = \int \limits_{E_0}^{E_l} \frac{Q_l^A(E)}{dE/d\rho s} dE$

This integral is solved analytically with the equations below.

Bethe [10] have a formula for $dE/d\rho s$, but this is valid for >30 keV. PAP use a formula valid for 1-50 keV:

$dE/d\rho s = -M/J \cdot 1/f(V)$

where



- $M = \sum \limits_{i} \frac{C_i Z_i}{A_i}$


- $ J = \exp(\sum \limits_{i} \frac{C_i Z_i}{A_i} \cdot \ln(J_i)/M)$
    - $J_i = 10^{-3} \cdot Z_i (10.04 + 8.25 \exp(\frac{-Z_i}{11.22}))$


- $ f(V) = \sum \limits_{k=1}^{3} D_k \cdot V^{P_k}$

    - $ V = \frac{E}{J}$
    - $D_1 = 6.6 \cdot 10^{-6}$
    - $P_1 = 0.78$
    - $D_2 = 1.12 \cdot 10^{-5}(1.35 - 0.45 J^2)$
    - $P_2 = 0.1$
    - $D_3 = \frac{2.2 \cdot 10^{-6}}{J}$
    - $P_3 = -(0.5-0.25J)$
    
i.e.

- $f(V) = 6.6 \cdot 10^{-6}\cdot V^{0.78} + \\ 
1.12\cdot 10^{-5}\cdot (1.35 - 0.45 J^2) \cdot V^{0.1} + \\
\frac{2.2 \cdot 10^{-6}}{J} \cdot V^{-(0.5-0.25J)}$


Calculate M, then J, then f(V) and then $dE/d\rho s$

- $dE/d\rho s$ in keV cm^2/g
- Ci is concentration of element i in the sample
- Zi is the atomic number of element i
- Ai is the atomic weight of element i 
    - I do not remember what i meant my this: "(OBS! somehow PAP ends with kV*cm^2/g, and not cm^3)"
- J is the mean ionization potential of the sample
- V is E/J, in keV
- M is the mean atomic mass of the sample, in g/cm^3

In [ ]:
# constants are calculated once
# list are list of const
# the "*" argument forces the user to use the keyword argument

In [ ]:
def calculate_dE_drhos(*, m: float, j: float, f_of_v: float):
    """Calculate the deceleration of the electron beam, dE/d rho s.
    Does the calculation for a single energy value."""
    return m / j / f_of_v
#   the PAP paper have a minus, but my plot is inversed with the minus
#   return (- m / j / f_of_v)

In [ ]:
def calculate_f_of_v(*, e: float, j: float):
    """Calculate the function f(v) = f(e/v) of the PAP algorithm.
    e: energy of the electron beam
    j: mean ionization potential of the material"""
    v = e / j
    return (6.6e-6 * v**0.78 + 
            1.12e-5 * (1.35 - 0.45 * j**2) * v**0.1 + 
            2.2e-6 / j * v**(-0.5 + 0.25 * j))

In [ ]:
def calculate_m(*, list_C: list, list_Z: list, list_A: list):
    """Calculate the mean atomic mass of the material, from the lists of atomic info"""
    # $M = \sum \limits_{i} \frac{C_i Z_i}{A_i}$
    m = 0
    for i in range(len(list_C)):
        m += list_C[i] * list_Z[i] / list_A[i]
    return m

In [ ]:
def calculate_j_i(*,z_i: float):
    """Calculate the ionization potential of one element, j_i"""
    # $J_i = 10^{-3} \cdot Z_i (10.04 + 8.25 \exp(\frac{-Z_i}{11.22}))$
    return z_i * 1e-3 * (10.04 + 8.25 * np.exp(-z_i / 11.22))

def calculate_mean_j(*, list_C: list, list_Z: list, list_A: list):
    """Calculate the mean ionization potential of the material, J"""
    # $ J = \exp(\sum \limits_{i} \frac{C_i Z_i}{A_i} \cdot \ln(J_i)/M)$
    list_Ji = []
    for z in list_Z:
        list_Ji.append(calculate_j_i(z_i=z))
    m = calculate_m(list_C=list_C, list_Z=list_Z, list_A=list_A)
    j = 0
    for i in range(len(list_C)):
        j += list_C[i] * list_Z[i] / list_A[i] * np.log(list_Ji[i]) / m
    return np.exp(j)

In [ ]:
# putting it together to calculate dE/d rho s for a list of elements at a given energy
def calculate_dE_drhos_whole(*, list_C: list, list_Z: list, list_A: list, e: float):
    """Calculate dE/d rho s for a list of elements at a given energy"""
    # $dE/d \rho s = \frac{m}{J} \cdot \frac{1}{f(e/v)}$
    m = calculate_m(list_C=list_C, list_Z=list_Z, list_A=list_A)
    j = calculate_mean_j(list_C=list_C, list_Z=list_Z, list_A=list_A)
    f_of_v = calculate_f_of_v(e=e, j=j)
    return calculate_dE_drhos(m=m, j=j, f_of_v=f_of_v)

In [ ]:
def test_dE_drhos(elements: list, concentrations: list, energy: float = None, energies: list = None):
    """Calculate dE/d rho s for a list of elements at a given energy or a list of energies"""
    list_C = concentrations
    list_Z = []
    list_A = []
    for element in elements:
        list_Z.append(hs.material.elements[element].General_properties['Z'])
        list_A.append(hs.material.elements[element].Physical_properties['density (g/cm^3)'])
    m = calculate_m(list_C=list_C, list_Z=list_Z, list_A=list_A)
    j = calculate_mean_j(list_C=list_C, list_Z=list_Z, list_A=list_A)

    print("Element\t\tZ\tA\tC")
    elements_info = ''
    for i in range(len(elements)):
        elements_info += f"{elements[i]}\t\t{list_Z[i]}\t{list_A[i]}\t{list_C[i]}\n"
    print(elements_info)
    print(f"M = {m:.4e}, J = {j:.4e}")

    if energy is not None:
        f_of_v = calculate_f_of_v(e=energy, j=j)
        return calculate_dE_drhos(m=m, j=j, f_of_v=f_of_v)
    elif energies is not None:
        f_of_v = [calculate_f_of_v(e=e, j=j) for e in energies]
        dE_drhos = np.array([calculate_dE_drhos(m=m, j=j, f_of_v=f) for f in f_of_v])
        return dE_drhos
    else:
        print("Please provide an energy (energy) or a list of energies (energies)")

In [ ]:
# testing on GaAs
elements = ['Ga','As']
list_C = [.5,.5]
energies = np.linspace(0.01, 50, 10000)
de_drhos = test_dE_drhos(elements=elements, concentrations=list_C, energies=energies)

elements = ['Ga','Sb']
list_C = [.5,.5]
energies = np.linspace(0.01, 50, 10000)
de_drhos2 = test_dE_drhos(elements=elements, concentrations=list_C, energies=energies)


In [ ]:
fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"))
fig.add_trace(go.Scatter(x=energies, y=de_drhos2, name='GaSb'))
fig.add_trace(go.Scatter(x=energies, y=de_drhos, name='GaAs'))

fig.update_layout(xaxis_title="Energy [keV]", #title="dE/d\u03C1s for electrons in GaAs", 
                  yaxis_title="dE/d\u03C1s [keV cm<sup>2</sup>/g]",
                  margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,
                  legend=dict(x=0.995, y=0.99, xanchor='right',))
fig.update_xaxes(type="log")

# fig.write_image('../mastersthesis/figures/PAP_deceleration_of_electrons.pdf')

#### Stopping power through ionization cross section

expressions that were proposed are generally too mathematically complex to allow for an
analytical calculation of the integral (3). A satisfactory way of varying the cross section
with U is obtained with the expression proposed by Hutchins [23]:

$ Q_l^A(U) \propto ln(U) / (U^m \cdot E_c^2)$

- with $ U = E/E_c $ as the overvoltage, where $E_c$ is the critical ionization energy.
- m is a constant dependent on the line type (K, L or M)
    - K-lines: m = 0.9, as suggested by Bastin (1998, PROZA96)
    - L-lines: m = 0.82
    - M-lines: m = 0.78

*Bastin, Dijkstra and Heijligers (1998) improved the m-coefficient in the Q-equation. They have other numbers for C, N and O, which have low Z.

https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/pdf/10.1002/%28SICI%291097-4539%28199801/02%2927%3A1%3C3%3A%3AAID-XRS227%3E3.0.CO%3B2-L

In [ ]:
def q_ionization_cross_section(*, e0: float, e_c: float, m_small: float):
    """Gives the ionization cross section for a given e0, e_c, and m_small
    Q = ln(U)/(u^m * e_c^2)"""
    u = e0 / e_c
    return np.exp(np.log(u) / (u**m_small * e_c**2))

def q_ionization_cross_section_2(*, e0: float, line: str):
    """Gives the Q for a given e0 and line"""
    e_c = theoretical_energy(line=line)
    line_type = line.split('_')[1][0]
    if line_type == 'K':
        m_small = 0.9
    elif line_type == 'L':
        m_small = 0.82
    else:
        raise ValueError(f'Line type {line_type} in {line} not supported.')
    u = e0 / e_c
    return np.exp(np.log(u) / (u**m_small * e_c**2))

$ 1/S = \int \limits_{E_0}^{E_C} (Q_C^A(E) \frac{dE}{d\rho s}) dE $

Using

$ T_k = 1 + P_k - m $ in 

$ 1/S = \frac{U_0}{V_0 \cdot M} \sum \limits_{k=1}^{3} D_k \cdot (V_0/U_0)^{P_k} \cdot ((T_k)U_0^{T_k} \cdot \ln(U_0)-U_0^{T_k}+1)/T_k^2 $


Which translates to

$1/S = \frac{U_O}{V_0 \cdot M} \cdot \\
(6.6 \cdot 10^{-6} (V_0/U_0)^{0.78} \cdot ((1+0.78-m)U_0^{1+0.78-m} \cdot \ln(U_0)-U_0^{1+0.78-m}+1)/(1+0.78-m)^2) + \\
((1.12 \cdot 10^{-5}(1.35-0.45J^2)) (V_0/U_0)^{0.1} \cdot ((1+0.1-m)U_0^{1+0.1-m} \cdot \ln(U_0)-U_0^{1+0.1-m}+1)/(1+0.1-m)^2) + \\
(2.2 \cdot 10^{-6}/J (V_0/U_0)^{(-0.5 +0.25J)} \cdot ((1+(-0.5 +0.25J)-m)U_0^{1+(-0.5 +0.25J)-m} \cdot \ln(U_0)-U_0^{1+(-0.5 +0.25J)-m}+1)/(1+(-0.5 +0.25J)-m)^2)
$

In [ ]:
# m_big is the mean atomic mass M
# m is the m in $T_k = 1+ P_k - m$
def calculate_one_over_S(*, u0: float, e_c: float, j: float, m_big: float, m_small: float):
    """ Calculates 1/S for a given energy and material
    
    Parameters
    ----------
    u0 : float
        Overvoltage
    e_c : float
        Critical ioniztion energy
    j : float
        Mean ionization potential
    m_big : float
        Mean atomic mass
    m : float
        K-, L-, or M-shell constant (0.9, 0.82, or 0.78)
    """
    v0 = e_c / u0
    return (u0/(v0 * m_big) * (
            # (6.6  10^{-6} (V_0/U_0)^{0.78}  ((1+0.78-m_small)U_0^{1+0.78-m}  \ln(U_0)-U_0^{1+0.78-m}+1)/(1+0.78-m_small)^2) + \\
            6.6e-6 * (v0/u0)**.78 * ((1+.78-m_small)*u0**(1+.78-m_small) * np.log(u0) - u0**(1+.78-m_small) + 1) / (1+.78-m_small)**2 + 
            # ((1.12  10^{-5}(1.35-0.45J^2)) (V_0/U_0)^{0.1}  ((1+0.1-m_small)U_0^{1+0.1-m}  \ln(U_0)-U_0^{1+0.1-m}+1)/(1+0.1-m_small)^2) + \\
            (1.12e-5 * (1.35-0.45*j**2)) * (v0/u0)**.1 * ((1+.1-m_small)*u0**(1+.1-m_small) * np.log(u0) - u0**(1+.1-m_small) + 1) / (1+.1-m_small)**2 +
            # (2.2  10^{-6}/J (V_0/U_0)^{(-0.5 +0.25J)}  ((1+(-0.5 +0.25J)-m_small)U_0^{1+(-0.5 +0.25J)-m}  \ln(U_0)-U_0^{1+(-0.5 +0.25J)-m}+1)/(1+(-0.5 +0.25J)-m_small)^2)
            (2.2e-6/j) * (v0/u0)**(-.5 + 0.25*j) * ((1+(-.5 + 0.25*j)-m_small)*u0**(1+(-.5 + 0.25*j)-m_small) * np.log(u0) - u0**(1+(-.5 + 0.25*j)-m_small) + 1) / (1+(-.5 + 0.25*j)-m_small)**2 
            ))

In [ ]:
def get_C_A_Z_lists(elements: list, concentrations: list):
    """Returns a list of C, A, and Z for a list of elements and concentrations"""
    list_C = concentrations
    list_Z = []
    list_A = []
    for element in elements:
        list_Z.append(hs.material.elements[element].General_properties['Z'])
        list_A.append(hs.material.elements[element].Physical_properties['density (g/cm^3)'])
    return list_C, list_A, list_Z

In [ ]:
##### Examples for 1/S
# GaAs, Ga_La with e_c = 1.1 keV, e0 = 15 keV
list_C, list_A, list_Z = get_C_A_Z_lists(elements=['Ga','As'], concentrations=[.5,.5])
m_big = calculate_m(list_C=list_C, list_Z=list_Z, list_A=list_A)
j = calculate_mean_j(list_C=list_C, list_Z=list_Z, list_A=list_A)
print("1/S : ", calculate_one_over_S(u0=15, e_c=1.1, j=j, m_big=m_big, m_small=0.82))


## Backscatter Loss Factor R, Appendix 1 in PAP


$ R = 1- \bar{\eta}  \cdot \bar{W} \cdot (1-G(U_0)) $

where

- $ \bar{W} = \bar{E}_r/E_0 = 0.595 + \bar{\eta}/3.7 + \bar{\eta}^{4.55} $

- $ \bar{\eta} = 1.75 \cdot 10^{-3} \cdot \bar{Z}_b + 0.37(1-\exp(-0.015\bar{Z}_b^{1.3})) $

- $\bar{Z}_b = (\sum C_i \cdot Z_i^{0.5})^2$

- $ G(U_0) = (U_0 - 1 - (1- \frac{1}{U_0^{1+q}})/(1+q)) / ((2+q)\cdot J(U_0))$

- $ J(U_0) = 1 + U_0 \cdot (\ln(U_0)-1) $

- $ q = (2 \bar{W} - 1) / (1 - \bar{W}) $




- R is the backscatter loss factor
- $\bar{\eta}$ is the mean backscattering coefficient
- $\bar{W}$ is 
- $\bar{Z}_b$ is the mean atomic number of the backscattered electrons, weighted
- $G(U_0)$ is from Coulon and Zeller (28)
- $U_0$ is the overvoltage, $E_0/E_c$

In [ ]:
def calculate_W(*, eta: float):
    return 0.595 + eta / 3.7 + eta**4.55

def calculate_eta(*, zb: float):
    return 1.75e-3 * zb + 0.37 * (1 - np.exp(-0.015 * zb**1.3))

def calculate_zb(*, list_C: list, list_Z: list):
    sum_Zb = 0
    for i in range(len(list_C)):
        sum_Zb += list_C[i] * list_Z[i]**0.5
    return sum_Zb**2

def calculate_j_of_u0(*, u0: float):
    return 1 + u0 * (np.log(u0) - 1)

def calculate_q(*, w: float):
    return (2 * w - 1) / (1 - w)

def calculate_g_of_u0(*, u0: float, j: float, q: float):
    return (u0 - 1 - (1 - 1 / u0**(1 + q)) / (1 + q)) / ((2 + q) * j)

In [ ]:
# $ R = 1- \bar{\eta}  \cdot \bar{W} \cdot (1-G(U_0)) $
def calculate_R(*, u0: float, list_C: list, list_Z: list):
    zb = calculate_zb(list_C=list_C, list_Z=list_Z)
    eta = calculate_eta(zb=zb)
    w = calculate_W(eta=eta)
    j = calculate_j_of_u0(u0=u0)
    q = calculate_q(w=w)
    g = calculate_g_of_u0(u0=u0, j=j, q=q)
    return 1 - eta * w * (1 - g)

In [ ]:
# calculate R for all Z
list_U = [1.1, 1.3, 1.5, 2, 3, 5, 20]
list_all_Z = np.arange(1,90)
list_all_A = np.ones_like(list_all_Z)
list_all_R = []
for z in list_all_Z:
    list_C = [1]
    list_Z = [z]
    list_R = []
    for u in list_U:
        list_R.append(calculate_R(u0=u, list_C=list_C, list_Z=list_Z))
    list_all_R.append(list_R)    
list_all_R = np.array(list_all_R)

# plot R for all Z
fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"))

for i, u0 in enumerate(list_U):
    fig.add_trace(go.Scatter(x=list_all_Z, y=list_all_R[:,i], name=f"U<sub>0</sub>={u0}"))

fig.update_layout(xaxis_title="Z", yaxis_title="Backscattering factor R") 
# title="Backscatter factor R for different Z with the PAP model", 
# tight layout
fig.update_layout(margin=dict(l=5, r=5, b=5, t=5), width=800, height=400)

# fig.write_image('../mastersthesis/figures/PAP_backscattering_factor.pdf')

fig

In [ ]:
# test on GaAs, Ga_La with e_c = 1.1 keV, e0 = 15 keV
list_C, list_A, list_Z = get_C_A_Z_lists(elements=['Ga','As'], concentrations=[.5,.5])
e_c = hs.material.elements['Ga'].Atomic_properties.Xray_lines.La['energy (keV)']
e0 = 15
u = e0 / e_c
j = calculate_mean_j(list_C=list_C, list_Z=list_Z, list_A=list_A)
m_big = calculate_m(list_C=list_C, list_Z=list_Z, list_A=list_A)
m = 0.82

one_over_s = calculate_one_over_S(u0=u, e_c=e_c, j=j, m_big=m_big, m_small=m)
r = calculate_R(u0=u, list_C=list_C, list_Z=list_Z)

print("1/S : ", one_over_s)
print("R : ", r)
print("1/S * R : ", one_over_s * r)

In [ ]:
e0_list = [5, 10, 15, 30]
print("e0\t\t1/S\t\t\tR\t\t\tR/S\t\t\tQ(E)\t\t\tR/S * Q(E)")
for e00 in e0_list:
    u = e00 / e_c
    one_over_s = calculate_one_over_S(u0=u, e_c=e_c, j=j, m_big=m_big, m_small=m)
    r = calculate_R(u0=u, list_C=list_C, list_Z=list_Z)
    q = q_ionization_cross_section(e0=e00, e_c=e_c, m_small=m)
    # print(f"e0 = {e00} keV, 1/S * R = {(one_over_s * r):.2e}, R/S * Q(E) = {r * one_over_s * q}")
    print(f"{e00}\t\t{one_over_s:.2e}\t\t{r:.2e}\t\t{(one_over_s * r):.2e}\t\t{q:.2e}\t\t{(one_over_s * r * q):.2e}")

The average number of primary ionizations due to an electron of initial energy E0 at
the level/ of the atoms A is commonly expressed by means of a deceleration factor liS and
a backscatter factor R

$n_A = C_A \cdot \frac{N^0}{A} \cdot \frac{R}{S} $

This is PER ELECTRON. Then I think N^0 cancel out, and I need C_A, A, R, and S (i.e. do not need the current).

In [ ]:
m_k = 0.9
m_l = 0.82
e0_max = 30

def add_line(fig, e_c, name, m):
    e0_list = np.arange(e_c, e0_max, 0.01)
    u_list = e0_list / e_c
    q_list = [q_ionization_cross_section(e0=e0, e_c=e_c, m_small=m) for e0 in e0_list]
    print('max q =', round(max(q_list),3), ' at U0 =' , u_list[q_list.index(max(q_list))], 
    ' at E =', e0_list[q_list.index(max(q_list))])
    fig.add_trace(go.Scatter(x=u_list, y=q_list, name=f'{name}, E<sub>C</sub>{e_c:.2f} keV'))

fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"))
add_line(fig, theoretical_energy('Ga_La'), 'Ga L\u03B1', m_l)
# add_line(fig, theoretical_energy('Ga_Ka'), 'Ga_Ka', m_k)
add_line(fig, theoretical_energy('As_La'), 'As L\u03B1', m_l)
# add_line(fig, theoretical_energy('As_Ka'), 'As_Ka', m_k)
# add_line(fig, theoretical_energy('Sb_La'), 'Sb_La', m_l)
# title='Ionization cross section as a function of overvolatge',
fig.update_layout( xaxis_title='U = E<sub>0</sub>/E<sub>c</sub>', yaxis_title='Q(U)',
                  margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,
                  legend=dict(x=0.995, y=0.99, xanchor='right',),
                  xaxis_range=[-0.1, 25])

# Annotate "Max Q = 1.45 at U = 3.38, i.e. E = 3.72 keV for Ga La"
fig.add_annotation(x=3.38, y=1.45, text="Max Q(U=3.38) = 1.45, i.e. E = 3.72 keV", ax=300, ay=0)
fig.add_annotation(x=3.38, y=1.314, text="Max Q(U=3.38) = 1.31, i.e. E = 4.34 keV", ax=250, ay=130)

# fig.write_image('../mastersthesis/figures/PAP_ionization_cross_section.pdf')


fig 

## Calculating F

$ F = (R/S) \cdot Q_l^A(E_0) $


In [ ]:
def calculate_F(*, elements: list, line: str, concentrations: list, e0: float):
    list_C, list_A, list_Z = get_C_A_Z_lists(elements=elements, concentrations=concentrations)
    e_c = theoretical_energy(line=line)
    line_type = line.split('_')[1][0]
    if line_type == 'K':
        m_small = 0.9
    elif line_type == 'L':
        m_small = 0.82
    else:
        raise ValueError(f'Line type {line_type} in {line} not supported.')
    u = e0 / e_c
    j = calculate_mean_j(list_C=list_C, list_Z=list_Z, list_A=list_A)
    m_big = calculate_m(list_C=list_C, list_Z=list_Z, list_A=list_A)
    s_inv = calculate_one_over_S(u0=u, e_c=e_c, j=j, m_big=m_big, m_small=m_small)
    r = calculate_R(u0=u, list_C=list_C, list_Z=list_Z)
    q = q_ionization_cross_section(e0=e0, e_c=e_c, m_small=m_small)

    return r*s_inv*q



print('F Ga_La: ' , calculate_F(elements=['Ga','As'], line='Ga_La', concentrations=[.5,.5], e0=15))
print('F As_La: ' , calculate_F(elements=['Ga','As'], line='As_La', concentrations=[.5,.5], e0=15))


In [ ]:
103207.0 / 0.005740332662198292

In [ ]:
61699.0 / 0.0027206947358069536

## Surface ionization $\phi(0)$

$\phi(0) = 1 + 3.3 (1-1/U_0^r)* \bar{\eta}^{1.2} $

with $ r = 2 - 2.3 \bar{\eta} $

(The exponent $\bar{\eta}^{1.2}$ kinda looks like $\bar{\eta}^{1 \cdot 2}$, but it is 1.2 that gives the same values as the plot in figure 24.)

In [ ]:
def phi_zero__surface_ionization(*, u: float, eta:float):
    return 1 + 3.3 * (1-1 / (u**(2 - 2.3 * eta))) * eta**1.2

# alternatively
def phi_zero__surface_ionization_2(*, e0: float, e_c: float, list_C: list, list_Z: list):
    zb = calculate_zb(list_C=list_C, list_Z=list_Z)
    eta = calculate_eta(zb=zb)
    u = e0 / e_c
    return 1 + 3.3 * (1 - 1 / (u**(2 - 2.3 * eta))) * eta**1.2

In [ ]:
def plot_phi_zero():
    # remaking plot 24 from the paper
    # use the second function
    e0_list = [1.1, 1.3, 1.5, 2, 3, 5, 20, 100]
    e_c = 1 # one to get the overvoltages in the paper
    all_z = np.arange(1, 90, 1)
    list_C = [1]
    fig = go.Figure()
    for e0 in e0_list:
        y = [phi_zero__surface_ionization_2(e0=e0, e_c=e_c, list_C=list_C, list_Z=[z]) for z in all_z]
        fig.add_trace(go.Scatter(x=all_z, y=y, name=f'{e0}'))
    return fig

fig = plot_phi_zero()
fig.update_layout(title='Surface ionization potential, \u03C6(0)', 
                    xaxis_title='Z', yaxis_title='\u03C6(0)',
                    margin=dict(l=5, r=5, b=5, t=50), width=600, height=400,
                    legend=dict(traceorder='reversed', title='Overvoltage'))

fig


# Mean depth of ionization, $\bar{R}$

Equation 28 in PAP

$ F / \bar{R} = 1 + [X \cdot \ln(1 + Y \cdot (1 - 1/U_0^{0.42}))]/\ln(1 + Y) $

where

- $ X = 1 + 1.3 \ln(\bar{Z}_b) $
- $ Y = 0.2 + \bar{Z}_b/200 $


i.e.

$ \bar{R} = F / (1 + [X \cdot \ln(1 + Y \cdot (1 - 1/U_0^{0.42}))]/\ln(1 + Y)) $

In [ ]:
def r_bar__average_depth_of_ionization(*, f: float, u: float, z_b: float):
    x = 1 + 1.3 * np.log(z_b)
    y = 0.2 + z_b/200
    # $ \bar{R} = F / (1 + [X \cdot \ln(1 + Y \cdot (1 - 1/U_0^{0.42}))]/\ln(1 + Y)) $
    return f / (1 + (x * np.log(1 + y * (1 - 1/u**0.42))) / np.log(1 + y))

In [ ]:
list_C, list_A, list_Z = get_C_A_Z_lists(elements=['Ga','As'], concentrations=[.5,.5])
f = calculate_F(elements=['Ga','As'], line='Ga_La', concentrations=list_C, e0=15)
zb = calculate_zb(list_C=list_C, list_Z=list_Z)
r_bar = r_bar__average_depth_of_ionization(f=f, u=15/theoretical_energy('Ga_La'), z_b=zb)

print(f'Average depth of ionization: {r_bar:.2e}')
print('As of now i do not know the units of this value. \nHowever, its scales nicely with E0')

In [ ]:
f/r_bar

## adjustments for high $\bar{R}$

"If $ F/\bar{R} < \phi(0) $, then $ \bar{R} = F/\phi(0) $"

# Initial slope $P$

equation 29


$ P = g \cdot h^4 \cdot F/\bar{R}^2 $

where

- $ g = 0.22 \ln(4 \bar{Z}_b) \cdot [1 - 2 \exp(-\bar{Z}_b \frac{U_0 - 1}{15})] $

- $ h = 1 - 10(1-\frac{1}{1+ U_0/10})/\bar{Z}_b^2 $


In [ ]:
# def slope_P(*, f, r_bar, zb, u)
def slope_P(*, f: float, r_bar: float, zb: float, u: float):
    g = 0.22 * np.log(4 * zb) * (1 - 2 * np.exp(-zb * (u - 1)/15))
    h = 1 - 10 * (1 - 1/(1 + u/10)) / zb**2
    return g * h**4 * f / r_bar**2

### Comment to the value $ g \cdot h^4 $ in the paper

"If necessary, limit the value $ g \cdot h^4 $ to the value $ 0.9 \cdot b\cdot  \bar{R}^2 \cdot [b - 2 \phi(0)/F] $"

I do not think that is necessary.

# Calculate b, a and $\epsilon$

<!-- $ b = \sqrt{2} \cdot (1 + \sqrt{1 - \bar{R} \cdot \phi(0) / F}/\bar{R}) $ -->
Wrote b wrong the first time

$ b = \sqrt{2} \cdot (1 + \sqrt{1 - \bar{R} \cdot \phi(0) / F})/\bar{R} $


$ a = [P + b \cdot (2\phi(0) - b \cdot F)] / [b \cdot F \cdot (2 - b \bar{R}) - \phi(0)] $

$ \epsilon = \frac{a-b}{b}  $

"If necessary, impose on $\epsilon$ a minimum absolute value (e.g. $10^{-6}$), and then assume $ a = b \cdot(1+e) $" (I do not think that is necessary.)

In [ ]:
def factor_small_b(*, r_bar: float, big_f: float, phi_zero: float):
    return np.sqrt(2) * (1 + np.sqrt(1 - r_bar * phi_zero / big_f)) / r_bar

def factor_small_a(*, p:float, b: float, big_f: float, phi_zero: float, r_bar: float):
    return (p + b * (2 * phi_zero - b * big_f)) / (b * big_f * (2 - b * r_bar) - phi_zero)

def factor_epsilon(*, a: float, b: float):
    return (a - b) / b

# Calculate B and A

$ B = [b^2 \cdot F \cdot (1 + \epsilon) - P - \phi(0) \cdot b \cdot (2+\epsilon) ] / \epsilon $

$ A = [B/b + \phi(0) - b \cdot F] \cdot \frac{1+ \epsilon}{\epsilon} $

In [ ]:

# $ B = [b^2 \cdot F \cdot (1 + \epsilon) - P - \phi(0) \cdot b \cdot (2+\epsilon) ] / \epsilon $
def factor_big_b(*, b: float, big_f: float, epsilon: float, p: float, phi_zero: float):
    return (b**2 * big_f * (1 + epsilon) - p - phi_zero * b * (2 + epsilon)) / epsilon

# $ A = [B/b + \phi(0) - b \cdot F] \cdot \frac{1+ \epsilon}{\epsilon} $
def factor_big_a(*, b: float, big_f: float, epsilon: float, phi_zero: float, big_b: float):
    return (big_b / b + phi_zero - b * big_f) * (1 + epsilon) / epsilon

# Emergent intensity, through $ F (\chi) $

" The fluorescent yield, the weights of the line of interest and the instrumental factors (solid angle and detection efficiency, incident current) are other factors omitted in eq (19) " (PAP p. 38)


$ I_A \propto C_A \cdot Q_l^A(E_0) \cdot F(\chi) $

where


$ \chi = \mu _\rho \cdot \cosec(TOA)$

$ F(\chi) = [\phi(0) + B/(b + \chi) - A \cdot b \cdot \epsilon / (b \cdot (1+\epsilon) + \chi)] / (b + \chi) $

or

$ F(\chi) = \frac{A}{a+ \chi} + \frac{\phi(0) - A}{b + \chi} + \frac{B}{(b + \chi)^2} $

which are the same.



In [ ]:
# $ F(\chi) = [\phi(0) + B/(b + \chi) - A \cdot b \cdot \epsilon / (b \cdot (1+\epsilon) + \chi)] / (b + \chi) $
def calc_f_of_chi(*, chi: float, phi_zero: float, big_b: float, b: float, big_a: float, epsilon: float):
    return (phi_zero + big_b / (b + chi) - big_a * b * epsilon / (b * (1 + epsilon) + chi)) / (b + chi)


# $ F(\chi) = \frac{A}{a+ \chi} + \frac{\phi(0) - A}{b + \chi} + \frac{B}{b^2 + \chi^2} $
# def calc_f_of_chi2(*, chi: float, phi_zero: float, big_b: float, b: float, big_a: float, a: float):
#     return big_a / (a + chi) + (phi_zero - big_a) / (b + chi) + big_b / ((b + chi)**2)

In [ ]:
def absorption_correction_factor(*, elements: list, concentrations: np.ndarray, line: str, e0: float):
    # gives the absorption correction factor for a given line and composition

    list_C, list_A, list_Z = get_C_A_Z_lists(elements=elements, concentrations=concentrations)
    u = e0 / theoretical_energy(line)
    mu_rho = hs.material.mass_absorption_mixture(elements=elements, weight_percent=concentrations*100, 
                                                energies=theoretical_energy(line))

    zb = calculate_zb(list_C=list_C, list_Z=list_Z)
    eta = calculate_eta(zb=zb)

    big_f = calculate_F(elements=elements, line=line, concentrations=list_C, e0=e0)

    phi_zero = phi_zero__surface_ionization(u=u, eta=eta)
    r_bar = r_bar__average_depth_of_ionization(f=big_f, u=u, z_b=zb)
    slope_p = slope_P(f=big_f, r_bar=r_bar, zb=zb, u=u)

    b = factor_small_b(r_bar=r_bar, big_f=big_f, phi_zero=phi_zero)
    a = factor_small_a(p=slope_p, b=b, big_f=f, phi_zero=phi_zero, r_bar=r_bar)
    epsilon = factor_epsilon(a=a, b=b)
    big_b = factor_big_b(b=b, big_f=big_f, epsilon=epsilon, p=slope_p, phi_zero=phi_zero)
    big_a = factor_big_a(b=b, big_f=big_f, epsilon=epsilon, phi_zero=phi_zero, big_b=big_b)

    chi = mu_rho / np.sin(np.deg2rad(35))  # cosecant of 35 degrees
    f_of_chi = calc_f_of_chi(chi=chi, phi_zero=phi_zero, big_b=big_b, b=b, big_a=big_a, epsilon=epsilon)


    # print('f1:', f_of_chi)

    return f_of_chi / big_f

In [ ]:
# testing the whole thing
# on GaAs, 15 kV, Ga_La and As_La

elements = ['Ga','As']
concentrations = np.array([.5,.5])
e0 = 15

ga_la_A = absorption_correction_factor(elements=elements, concentrations=concentrations, line='Ga_La', e0=e0)
as_la_A = absorption_correction_factor(elements=elements, concentrations=concentrations, line='As_La', e0=e0)

print(f'Ga_La: {ga_la_A:.2f}\nAs_La: {as_la_A:.2f}')


In [ ]:
# read a df at "../sem-eds-qc/output.csv"
df = pd.read_csv('../sem-eds-qc/output.csv')
df.head(3)

In [ ]:
def wt2at(wt1, wt2, atwt1, atwt2):
        # at%_1 = (wt%_1 / at_wt_1) / (wt%_1 / at_wt_1 + wt%_2 / at_wt_2)
        return (wt1 / atwt1) / (wt1 / atwt1 + wt2 / atwt2)

def calculate_atom_percent(wt_list, elements):
        atwt = [hs.material.elements[element].General_properties['atomic_weight'] for element in elements]
        # print(atwt)
        at = wt2at(wt_list[0], wt_list[1], atwt[0], atwt[1])
        print(f'{elements[0]} {at:.2f} {elements[1]} {1-at:.2f}')

calculate_atom_percent([0.45, 0.55], ['As', 'Ga'])
calculate_atom_percent([0.37, 0.63], ['Ga', 'Sb'])

# calculate_atom_percent([.93, 0.07], ['Fe', 'C'])

In [ ]:
df['xpp'] = 0.0

print('Vacc  Line\tA\tQ\twt%\twt%:A\tat%\tat%:A\tat%:AQ\tat%:q')

for xi in df['x'].unique():
    elements = df[df['x'] == xi]['Element'].unique()
    i1 = df[df['x'] == xi]['i'].to_list()[0]
    i2 = df[df['x'] == xi]['i'].to_list()[1]
    l1 = df[df['x'] == xi]['Line'].to_list()[0]
    l2 = df[df['x'] == xi]['Line'].to_list()[1]
    # print(elements, i1, i2, l1, l2)

    vacc = df[df['x'] == xi]['kV'].to_list()[0]

    acorr1 = absorption_correction_factor(elements=elements, concentrations=np.array([0.5, 0.5]), line=l1, e0=vacc)
    acorr2 = absorption_correction_factor(elements=elements, concentrations=np.array([0.5, 0.5]), line=l2, e0=vacc)

    to_percent = 100.0

    wt1 = to_percent * i1/(i1+i2)
    wt2 = to_percent * i2/(i1+i2)

    q1 = q_ionization_cross_section_2(e0=vacc, line=l1)
    q2 = q_ionization_cross_section_2(e0=vacc, line=l2)

    wt1_q = to_percent * (i1/q1) / ((i1/q1) + (i2/q2))
    wt2_q = to_percent * (i2/q2) / ((i1/q1) + (i2/q2))

    wt1_A = to_percent * i1/acorr1 / (i1/acorr1 + i2/acorr2)
    wt2_A = to_percent * i2/acorr2 / (i1/acorr1 + i2/acorr2)

    wt1_Aq = to_percent * i1/(acorr1 * q1) / (i1/(acorr1 * q1) + i2/(acorr2 * q2))
    wt2_Aq = to_percent * i2/(acorr2 * q2) / (i1/(acorr1 * q1) + i2/(acorr2 * q2))

    atwt = [hs.material.elements[element].General_properties['atomic_weight'] for element in elements]

    at1 = to_percent * wt2at(wt1, wt2, atwt[0], atwt[1])
    at2 = to_percent * wt2at(wt2, wt1, atwt[1], atwt[0])

    at1_q = to_percent * wt2at(wt1_q, wt2_q, atwt[0], atwt[1])
    at2_q = to_percent * wt2at(wt2_q, wt1_q, atwt[1], atwt[0])

    at1_A = to_percent * wt2at(wt1_A, wt2_A, atwt[0], atwt[1])
    at2_A = to_percent * wt2at(wt2_A, wt1_A, atwt[1], atwt[0])

    print(f'{int(vacc):<4}  {l1}\t{acorr1:.3f}\t{q1:.3f}\t{wt1:.0f}\t{wt1_A:.0f}\t{at1:.0f}\t{at1_A:.0f}\t{wt1_Aq:.0f}\t{at1_q:.0f}')
    # print(f'{int(vacc):<4}  {l2}\t{acorr2:.3f}\t{q2:.3f}\t{wt2:.0f}\t{wt2_A:.0f}\t{at2:.0f}\t{at2_A:.0f}\t{wt2_Aq:.0f}\t{at2_q:.0f}')



In [ ]:

hs.material.mass_absorption_mixture(elements=['As', 'Ga'], weight_percent=concentrations*100, energies=theoretical_energy('As_Ka'))

In [ ]:
hs.material.mass_absorption_mixture(elements=['Ga', 'Sb'], weight_percent=concentrations*100, energies=theoretical_energy('Sb_La'))


In [ ]:
61699.0/(q_ionization_cross_section(e0=15, e_c=theoretical_energy('As_La'), m_small=0.82) * .146)

$ I_A \propto C_A \cdot Q_l^A(E_0) \cdot F(\chi) $

where

- $ C_A $ is the concentration of element A
- $ Q_l^A(E_0) $ is the ionization cross section of element A at the energy $E_0$, which is the beam energy
- $ F(\chi) $ is the emergent intensity

# plotting $\phi (\rho z)$

$ \phi (\rho z) = A \cdot \exp(- a \cdot (\rho z)) + (B \cdot (\rho z) + \phi(0) - A) \cdot \exp(- b \cdot (\rho z)) $



In [ ]:
# $ \phi (\rho z) = A \cdot \exp(- a \cdot (\rho z)) + (B \cdot (\rho z) + \phi(0) - A) \cdot \exp(- b \cdot (\rho z)) $
def phi_rhoz(*, rhoz: np.array, A: float, a: float, B: float, b: float, phi_zero: float) -> np.array:
    return A * np.exp(- a * rhoz) + (B * rhoz + phi_zero - A) * np.exp(- b * rhoz)

In [ ]:
type(go.Figure)

In [ ]:
def phi_rhoz_2(*, elements: list, concentrations: np.ndarray, line: str, e0: float, rhoz: np.array):
    # gives the absorption correction factor for a given line and composition
    list_C, list_A, list_Z = get_C_A_Z_lists(elements=elements, concentrations=concentrations)
    u = e0 / theoretical_energy(line)
    mu_rho = hs.material.mass_absorption_mixture(elements=elements, weight_percent=concentrations*100, 
                                                energies=theoretical_energy(line))
    zb = calculate_zb(list_C=list_C, list_Z=list_Z)
    eta = calculate_eta(zb=zb)
    big_f = calculate_F(elements=elements, line=line, concentrations=list_C, e0=e0)
    phi_zero = phi_zero__surface_ionization(u=u, eta=eta)
    r_bar = r_bar__average_depth_of_ionization(f=big_f, u=u, z_b=zb)
    slope_p = slope_P(f=big_f, r_bar=r_bar, zb=zb, u=u)
    b = factor_small_b(r_bar=r_bar, big_f=big_f, phi_zero=phi_zero)
    a = factor_small_a(p=slope_p, b=b, big_f=f, phi_zero=phi_zero, r_bar=r_bar)
    epsilon = factor_epsilon(a=a, b=b)
    big_b = factor_big_b(b=b, big_f=big_f, epsilon=epsilon, p=slope_p, phi_zero=phi_zero)
    big_a = factor_big_a(b=b, big_f=big_f, epsilon=epsilon, phi_zero=phi_zero, big_b=big_b)
    chi = mu_rho / np.sin(np.deg2rad(35))  # cosecant of 35 degrees
    f_of_chi = calc_f_of_chi(chi=chi, phi_zero=phi_zero, big_b=big_b, b=b, big_a=big_a, epsilon=epsilon)

    # phi of (rho z)
    print(f'A: {big_a:>9.2e}, \ta: {a:>9.2e}, \tB: {big_b:>9.2e}, \tb: {b:>9.2e}, \t\u03C6(0): {phi_zero:>9.2e}, \t\u03B5: {epsilon:>9.2e}')
    phi = phi_rhoz(rhoz=rhoz, A=big_a, a=a, B=big_b, b=b, phi_zero=phi_zero)
    return phi


In [ ]:
fig = go.Figure()
rhoz = np.linspace(0, 0.01, 1000+1)
# phi = phi_rhoz_2(elements=['Ga', 'As'], concentrations=np.array([.5, .5]), line='Ga_La', e0=14, rhoz=rhoz)
# fig.add_trace(go.Scatter(x=rhoz, y=phi, mode='lines'))

phi_5 = phi_rhoz_2(elements=['Ga', 'As'], concentrations=np.array([.5, .5]), line='Ga_La', e0=5, rhoz=rhoz)
phi_10 = phi_rhoz_2(elements=['Ga', 'As'], concentrations=np.array([.5, .5]), line='Ga_La', e0=10, rhoz=rhoz)
phi_15 = phi_rhoz_2(elements=['Ga', 'As'], concentrations=np.array([.5, .5]), line='Ga_La', e0=15, rhoz=rhoz)

fig.add_trace(go.Scatter(x=rhoz, y=phi_5, mode='lines', name='5 keV'))
fig.add_trace(go.Scatter(x=rhoz, y=phi_10, mode='lines', name='10 keV'))
fig.add_trace(go.Scatter(x=rhoz, y=phi_15, mode='lines', name='15 keV'))

fig.update_layout(yaxis_title='\u03C6 (\u03C1z)', xaxis_title='\u03C1z [g/cm<sup>2</sup>]')

In [ ]:
# # phi_rhoz_plot(elements=['Cu', 'Al'], concentrations=np.array([.5, .5]), line='Cu_La', e0=6)

# fig = go.Figure()
# rhoz = np.linspace(0, 0.01, 1000+1)

# phi_5 = phi_rhoz_2(elements=['Cu', 'Al'], concentrations=np.array([.5, .5]), line='Cu_La', e0=5, rhoz=rhoz)
# phi_10 = phi_rhoz_2(elements=['Cu', 'Al'], concentrations=np.array([.5, .5]), line='Cu_La', e0=10, rhoz=rhoz)
# phi_15 = phi_rhoz_2(elements=['Cu', 'Al'], concentrations=np.array([.5, .5]), line='Cu_La', e0=15, rhoz=rhoz)

# fig.add_trace(go.Scatter(x=rhoz, y=phi_5, mode='lines', name='Cu_La 5 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_10, mode='lines', name='Cu_La 10 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_15, mode='lines', name='Cu_La 15 keV'))

# phi_5 = phi_rhoz_2(elements=['Cu', 'Al'], concentrations=np.array([.5, .5]), line='Ga_La', e0=5, rhoz=rhoz)
# phi_10 = phi_rhoz_2(elements=['Cu', 'Al'], concentrations=np.array([.5, .5]), line='Ga_La', e0=10, rhoz=rhoz)
# phi_15 = phi_rhoz_2(elements=['Cu', 'Al'], concentrations=np.array([.5, .5]), line='Ga_La', e0=15, rhoz=rhoz)

# fig.add_trace(go.Scatter(x=rhoz, y=phi_5, mode='lines', name='Ga_La 5 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_10, mode='lines', name='Ga_La 10 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_15, mode='lines', name='Ga_La 15 keV'))

# fig.update_layout(title='Something goes wrong using Cu_La at 15 kV', yaxis_range=[-.2, 3],)

In [ ]:
# # phi_rhoz_plot(elements=['C'], concentrations=np.array([2.2]), line='C_Ka', e0=4)

# fig = go.Figure()
# rhoz = np.linspace(0, 0.001, 1000+1)

# phi_5 = phi_rhoz_2(elements=['Ga', 'Sb'], concentrations=np.array([0.5, 0.5]), line='Sb_La', e0=5, rhoz=rhoz)
# phi_10 = phi_rhoz_2(elements=['Ga', 'Sb'], concentrations=np.array([0.5, 0.5]), line='Sb_La', e0=10, rhoz=rhoz)
# phi_15 = phi_rhoz_2(elements=['Ga', 'Sb'], concentrations=np.array([0.5, 0.5]), line='Sb_La', e0=15, rhoz=rhoz)
# phi_30 = phi_rhoz_2(elements=['Ga', 'Sb'], concentrations=np.array([0.5, 0.5]), line='Sb_La', e0=30, rhoz=rhoz)


# fig.add_trace(go.Scatter(x=rhoz, y=phi_5, mode='lines', name='Sb_La 5 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_10, mode='lines', name='Sb_La 10 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_15, mode='lines', name='Sb_La 15 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_30, mode='lines', name='Sb_La 30 keV'))

# phi_5 = phi_rhoz_2(elements=['Ga', 'Sb'], concentrations=np.array([0.5, 0.5]), line='Ga_La', e0=5, rhoz=rhoz)
# phi_10 = phi_rhoz_2(elements=['Ga', 'Sb'], concentrations=np.array([0.5, 0.5]), line='Ga_La', e0=10, rhoz=rhoz)
# phi_15 = phi_rhoz_2(elements=['Ga', 'Sb'], concentrations=np.array([0.5, 0.5]), line='Ga_La', e0=15, rhoz=rhoz)
# phi_30 = phi_rhoz_2(elements=['Ga', 'Sb'], concentrations=np.array([0.5, 0.5]), line='Ga_La', e0=30, rhoz=rhoz)


# fig.add_trace(go.Scatter(x=rhoz, y=phi_5, mode='lines', name='Ga_La 5 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_10, mode='lines', name='Ga_La 10 keV'))
# fig.add_trace(go.Scatter(x=rhoz, y=phi_15, mode='lines', name='Ga_La 15 keV'))
# # fig.add_trace(go.Scatter(x=rhoz, y=phi_30, mode='lines', name='Ga_La 30 keV'))


# # fig.update_layout(title='Something goes wrong using C_Ka in C', yaxis_range=[-200, 3],)
# # log x
# # fig.update_layout(title='Something goes wrong using Sb_La in GaSb', xaxis_type="log", xaxis_title='\u03C1z [g/cm<sup>2</sup>]')